The aim - is to develop a model that will give accurate predictions for the customer's test sample, but the training sample is not given -  to be collected

In [1]:
import pandas as pd
import bs4
import re
import numpy as np
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

# allows to open more files in the same time
import resource
resource.setrlimit(resource.RLIMIT_NOFILE, (10000,-1))
# resource.getrlimit(resource.RLIMIT_NOFILE)

In [2]:
# test.csv - Test data with revies
with open('test.csv', 'r') as file:
    a = file.read()
parser = bs4.BeautifulSoup(a, 'lxml')
test_data = [item.get_text(strip=True) for item in parser.find_all('review')]

Internet shop links have similar structure.
Here we are 1) parsing all necessary links, 2) parsing all feedbacks and number of star for each feedback (5* - is good, 1* - is bad)

In [3]:
%%writefile parse_mobile_links.py
import requests
import bs4
from multiprocessing import Pool
import codecs
import time
hdr = {"User-Agent": "Mozilla/5.0",
       "Accept-Encoding":"gzip, deflate, sdch",
       "Accept-Language":"ru-RU,ru;q=0.8,en-US;q=0.6,en;q=0.4"}

def parse_page(url):
    n = 0
    s = 0
    COND_ALLOW_PARS = False
    while not COND_ALLOW_PARS:
        text = requests.get(url, headers = hdr).text
        parser = bs4.BeautifulSoup(text, 'lxml')
        tag_h1 = parser.find('h1')
        COND_ALLOW_PARS = (tag_h1.get_text()[:10] == u'\u041c\u043e\u0431\u0438\u043b\u044c\u043d\u044b\u0435 ')
        print "trying to parse " + url
        print tag_h1
        print "COND_ALLOW_PARS :{}".format(COND_ALLOW_PARS)
        s += 5+n
        time.sleep(5+n)
        n += 1
        print "Sleep now: {} sec. Total sleep: {} sec".format(n+5, s)
    x = parser.findAll('div', attrs={'class':'citate'})
    z = [item.find('a').get('href') for item in x]
    print "parssed link" + url
    time.sleep(5)
    return z

p = Pool(1)
url_list = ['http://irecommend.ru/category/mobilnye-telefony?page=' + str(n) for n in range(0, 162)]
    
if __name__ == '__main__':    
    map_results = p.map(parse_page, url_list)
    reduce_results = reduce(lambda x,y: x + y, map_results)
    with codecs.open('parsing_results2.txt', 'w', 'utf-8') as output_file:
        print >> output_file, u'\n'.join(reduce_results)

Overwriting parse_mobile_links.py


In [4]:
with open('parsing_results2.txt', 'r') as file:
    l = file.read()
short_links = l.split('\n')
print 'Number of links: {}'.format(len(short_links))
links = ['http://irecommend.ru' + i for i in short_links]

Number of links: 3222


Data parsing

In [ ]:
#%%writefile parse_mobile_data.py
import requests
import bs4
#from multiprocessing import Pool
#import codecs
import time
#import random

with open('parsing_results2.txt', 'r') as file:
    l = file.read()
short_links = l.split('\n')
url_list = ['http://irecommend.ru' + i for i in short_links]

hdr = {"User-Agent": "Mozilla/5.0",
       "Accept-Encoding":"gzip, deflate, sdch",
       "Accept-Language":"ru-RU,ru;q=0.8,en-US;q=0.6,en;q=0.4"}

def parse_page(url):
    n = 0
    s = 0
    
    COND_ALLOW_PARS = True
    # sleep while banned
    while COND_ALLOW_PARS:
        text = requests.get(url, headers = hdr).text
        parser = bs4.BeautifulSoup(text, 'lxml')
        tag_h1 = parser.find('h1')
        COND_ALLOW_PARS = (tag_h1.get_text()[:28] == u'\u041d\u0430 \u0441\u0430\u0439\u0442\u0435 \u0432\u0435\u0434\u0443\u0442\u0441\u044f \u0442\u0435\u0445\u043d\u0438\u0447\u0435\u0441\u043a\u0438\u0435')
        #print "trying to parse " + url
        #print tag_h1
        #print "COND_ALLOW_PARS :{}".format(COND_ALLOW_PARS)
        s += 5+n
        time.sleep(5+n)
        n += 1
        #print "Sleep now: {} sec. Total sleep: {} sec".format(n+5, s)
    try:
        rating = float(parser.find('span', attrs={'class':'on rating', 'itemprop':'ratingValue'}).get_text())
    except:
        rating = -1
    try:
        text = parser.find('div', attrs={'class':'description hasinlineimage', 'itemprop':'reviewBody'}).get_text().replace('\n', "").replace('\t', '')
    except:
        try:
            # different class values
            text = parser.find('div', attrs={'class':'description', 'itemprop':'reviewBody'}).get_text().replace('\n', "").replace('\t', '')
        except:
            text = ""
    z = {'rating' : rating, 'feedback':text}
    #print "parssed link" + url
    sleep_s = 5 + random.randint(0,10)/10.0
    #print 'Sleep after parsing {} sec'.format(sleep_s)
    time.sleep(sleep_s)
    return z
ind = 0
data = list()
for url in url_list:
    res = parse_page(url)
    ind += 1
    data.append(res)
    print ind

Pre-process the text

In [589]:
import string

from nltk.stem.snowball import RussianStemmer
from nltk import word_tokenize

table = { ord(char): ord(' ') for char in string.punctuation }
stemmer = RussianStemmer() 
def preprocess(doc): 
    words = [stemmer.stem(t) for t in doc.lower().translate(table).split()]            
    return ' '.join(words)

train_data_X = [preprocess(item) for item in test_data]
data.feedback = [preprocess(item) for item in data.feedback]

In [590]:
# data_original
data = data_original[data_original.rating<>-1]

In [454]:
#feedback_cleaned = [" ".join(re.findall('[^0-9/.:,;/s)(~+-?!"]+', item)) for item in data.feedback.values]

In [456]:
#test_data_x = [" ".join(re.findall('[^0-9/.:,;/s)(~+-?!"]+', item)) for item in test_data]

In [ ]:
data['feedback_cleaned'] = feedback_cleaned

In [ ]:
a = 5
b = 2
def rating_cleaning(rat):
    if rat >= a:
        return 1
    elif rat<= b:
        return 0
    else:
        return -1
data['rating_cleaned'] = map(rating_cleaning, data.rating)

In [592]:
data.head()

,feedback,rating,rating_cleaned
0,"Всем привет!Начну с того, что я именно тот че...",4.0,-1
1,Купила смартфон Samsung Galaxy J7 более полуг...,4.0,-1
2,Изначально в семье появился один такой телефо...,5.0,1
3,Купила на Алиэкспресс телефон. На этот раз дл...,5.0,1
4,Классный телефончик без преувеличении могу ск...,5.0,1


In [ ]:
data1 = data[['feedback', 'rating_cleaned']]
#data1 = data[['feedback_cleaned', 'rating_cleaned']]
#data1.columns = ['feedback', 'rating', 'feedback_cleaned', 'rating_cleaned']
data1 = data1[data1.rating_cleaned <> -1]
print data1.rating_cleaned.value_counts()
print data1.head()

In [521]:
#data2 = data[['feedback', 'rating', 'feedback_cleaned', 'rating_cleaned2']]
#data2.columns = ['feedback', 'rating', 'feedback_cleaned', 'rating_cleaned']
#data2 = data2[data2.rating_cleaned <> -1]
#data2.rating_cleaned.value_counts()

In [594]:
#data for model development
X = data1.feedback.tolist()
#X = data1.feedback_cleaned.tolist()
y = data1.rating_cleaned.tolist()

Search for the best Classifier. DO NOT START - it takes a long time.
We use Vectorizer: CountVectorizer, TfidfVectorizer and models: SVC, SGD, logistic

In [260]:
SVC = LinearSVC()
SGD = SGDClassifier()
logistic = LogisticRegression()

stop = stopwords.words('english')

count_vec = CountVectorizer()
TFDF = TfidfVectorizer()
pipe_all = Pipeline(steps=[('count_vec', count_vec), ('clf', SVC)])
pipe_all.set_params(count_vec__min_df= 2.0, count_vec__max_df=0.99, count_vec__stop_words = 'english',
                   count_vec__ngram_range = (1,1), count_vec__analyzer = 'word',
                   clf__class_weight='balanced')

params = dict(
    count_vec = [count_vec, TFDF],
    count_vec__min_df = [0, 1, 2, 3],
    count_vec__max_df = [0.99, 0.98, 0.95, 1.0],
    count_vec__stop_words = ['english', stop, None],
    count_vec__analyzer = ['word'],# 'char_wb'],
    count_vec__ngram_range=[(1,1), (2,2), (1, 2), (2,3), (1,3), (3,3)],
    clf = [SVC, SGD, logistic],
    clf__class_weight=['balanced', None])

grid_search = GridSearchCV(pipe_all, param_grid=params, cv=3)
grid_search.fit(X, y)

mean_scores_test = np.array(grid_search.cv_results_['mean_test_score'])
result_test = mean_scores_test
print grid_search.best_estimator_.named_steps

{'clf': SGDClassifier(alpha=0.0001, average=False, class_weight='balanced',
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False), 'count_vec': TfidfVectorizer(analyzer='word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.99, max_features=None, min_df=1,
        ngram_range=(2, 2), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'w...n', u'ma', u'mightn', u'mustn', u'needn', u'shan', u'shouldn', u'wasn', u'weren

In [595]:
from stop_words import get_stop_words
stop_words = get_stop_words('russian')

The best result is on SVC classifier with TFidF. Cross validaiotn on 3 folds showed the result 91.2%

In [596]:
SVC = LinearSVC(class_weight='balanced', C=0.9)
SGD = SGDClassifier(class_weight='balanced')
logistic = LogisticRegression(class_weight='balanced', C=0.9, n_jobs=4)

estimator = SVC

TFDF = TfidfVectorizer(min_df= 2, max_df=0.99, stop_words = stop_words, ngram_range = (1,3), analyzer = 'word')
pipe_best1 = Pipeline(steps=[('count_vec', TFDF), ('clf', estimator)])
cross_val_score(pipe_best1, X, y, cv=3).mean()

0.91185888892442968

In [586]:
pipe_best1.fit(X,y)

Pipeline(steps=[('count_vec', TfidfVectorizer(analyzer='word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.99, max_features=None, min_df=2,
        ngram_range=(1, 3), norm=u'l2', preprocessor=None, smooth_idf...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [587]:
results = pipe_best1.predict(train_data_X)

In [588]:
# to save the results
z = ['pos' if item == 1 else 'neg' for item in results]
z = pd.DataFrame(z)
z.columns = ['y']
z.index.name = 'id'
file_name = 'pipe_SGD_TFDF1_tokenizer15.csv'
z.to_csv(path_or_buf=file_name)